In [ ]:
#Storing the texts in a list and using newspaper3k to get a summary

for link in links:
    article = Article(link.getText(), language="en")
    article.download()
    if article.download_state != 2:
        continue
    article.parse()
    article.nlp()
    print(article.title)
    print('-----------------------')
    print(article.text)
    print('$$$$$$$$$$$$$$$$$$$$$$$$$')
    print(article.summary)
    print('=====================')
   


In [41]:
from sqlalchemy import *
db = create_engine('sqlite:///HinduDB2.db', echo=True)

metadata = MetaData(db)

news = Table('news', metadata,
            Column('id', Integer, primary_key=True),
            Column('paperName', String(100)),
            Column('title', String(250)),
            Column('summary', String(10000))
            )
news.create()

2018-12-05 23:39:40,778 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-05 23:39:40,781 INFO sqlalchemy.engine.base.Engine ()
2018-12-05 23:39:40,783 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-05 23:39:40,784 INFO sqlalchemy.engine.base.Engine ()
2018-12-05 23:39:40,788 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE news (
	id INTEGER NOT NULL, 
	"paperName" VARCHAR(100), 
	title VARCHAR(250), 
	summary VARCHAR(10000), 
	PRIMARY KEY (id)
)


2018-12-05 23:39:40,789 INFO sqlalchemy.engine.base.Engine ()
2018-12-05 23:39:41,099 INFO sqlalchemy.engine.base.Engine COMMIT


In [42]:
from newspaper import Article
from bs4 import BeautifulSoup
import requests
import urllib

#Hindu Business Line RSS Feed
url = "https://www.thehindubusinessline.com/news/national/feeder/default.rss"

resp = requests.get(url)

bsObj = BeautifulSoup(resp.content, features = "xml")
links = bsObj.findAll('link', text=True)

##Links extracted from RSS Feed
print('EXTRACTED LINKS FROM RSS FEED')
for link in links:
    print(link.get_text())

EXTRACTED LINKS FROM RSS FEED
https://www.thehindubusinessline.com/
https://www.thehindubusinessline.com/news/national/walmart-to-open-50-stores-in-next-5-years/article25673732.ece
https://www.thehindubusinessline.com/news/national/maharashtra-ropes-in-49-firms-for-rural-change/article25673722.ece
https://www.thehindubusinessline.com/news/national/food-supplied-in-restaurant-cum-sweet-shop-to-be-treated-as-service-attract-5-gst-aar/article25673542.ece
https://www.thehindubusinessline.com/news/national/samsung-launches-mumbais-first-led-cinema-screen-with-inox/article25673304.ece
https://www.thehindubusinessline.com/news/national/hotel-association-threatens-protest-against-travel-portals/article25673272.ece
https://www.thehindubusinessline.com/news/national/blast-kills-employee-of-start-up-incubated-at-iisc/article25673232.ece
https://www.thehindubusinessline.com/news/national/owaisi-im-supporting-trs-as-regional-parties-can-make-a-difference/article25673225.ece
https://www.thehindubusi

In [43]:
#Using pagerank and textrank to get the summary

from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [44]:
import numpy as np
from nltk.tokenize import sent_tokenize
from operator import itemgetter 
import warnings
warnings.filterwarnings("ignore")

#Store all text in list
article_text = []
article_title = []
for link in links:
    article = Article(link.getText(), language="en")
    article.download()
    article.parse()
    if(len(article.text.split('.'))<=3):
        continue
    article_text.append(article.text)
    article_title.append(article.title)

stop_words = stopwords.words('english')

def build_similarity_matrix(sentences, stopwords=None):
    
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    
    for idx in range(len(S)):
        if(S[idx].sum()==0):
            break
        S[idx] /= S[idx].sum()
 
    return S

def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P


counter = 0
temp = ""
i=news.insert()
for texts in article_text:
    sentences = sent_tokenize(texts)
    S = build_similarity_matrix(sentences, stop_words)  
    sentence_ranks = pagerank(S)
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    SUMMARY_SIZE = 5
    SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
    summary = itemgetter(*SELECTED_SENTENCES)(sentences)
    # Print the actual summary
    print(article_title[counter])
    print('\nSUMMARY')
    print('------------------------------')
    for sentence in summary:
        temp=''.join(sentence)
        print(''.join(sentence))
    print()
    i.execute({'paperName':'Hindu Business Line', 'title':article_title[counter],'summary':temp})
    counter+=1
    

Walmart to open 50 stores in next 5 years

SUMMARY
------------------------------
50 new stores

Interacting with the media on the sidelines of State of Maharashtra’s Agribusiness and Rural Transformation (SMART) event, Iyer said the overall country plan is on track and the company has already signed up for 24 stores in the country.
He pointed out that the company currently has a strategy, which is more tuned towards tier-2 cities as it offers a good opportunity to serve the kirana stores as well as sourcing agri-products from the rural markets.
Iyer said that horticulture crops such as strawberry, pomegranate and grapes from Maharashtra is a great opportunity for the company and it has deployed manpower on the ground to help with capacity building for farmers.
World Bank project

SMART is a World Bank assisted project to revamp the agricultural value chain in the State with a focus on marginal farmers with small land holdings across 1,000 villages.
The remaining $10 million will come 

2018-12-05 23:39:59,632 INFO sqlalchemy.engine.base.Engine ('Hindu Business Line', 'Blast kills employee of start-up incubated at IISc', 'An FIR to this effect has been registered with the Sadhashivanagar police station and are investigating into the case.')
2018-12-05 23:39:59,636 INFO sqlalchemy.engine.base.Engine COMMIT
Owaisi: I’m supporting TRS as regional parties can make a difference

SUMMARY
------------------------------
Owaisi, All India Majlis-e-Ittehadul Muslimeen Chief, said, “I am supporting Telangana Rashtra Samithi (TRS) as I believe regional parties can better fulfil people’s aspirations.”

Speaking on various political issues in the elections to the Telangana Assembly on December 7, Owaisi saidhe was confident of the TRS’ return to power as it is better placed to address issues of people in the State.
Exuding confidence that AIMIM will win eight seats in Hyderabad, he said “The A team in Hyderabad will be successful.” Questioning both the Congress and BJP for attackin

2018-12-05 23:40:01,860 INFO sqlalchemy.engine.base.Engine ('Hindu Business Line', '254 projects sanctioned for Ganga rejuvenation: Gadkari', '“We will also be undertaking work in Madhya Pradesh,” the minister said referring to projects to be undertaken on a sub-tributary of Ganga.')
2018-12-05 23:40:01,886 INFO sqlalchemy.engine.base.Engine COMMIT
Domestic operations at Terminal 1 to be inaugurated on Dec 12

SUMMARY
------------------------------
Cochin International Airport Ltd (CIAL) said that domestic operations at the renovated Terminal-1 and the enhancement of the solar power production capacity to 40 MWp would be inaugurated on December 12.
Kerala’s Chief Minister Pinarayi Vijayan will inaugurate the two mega projects of the CIAL, which had handled one crore passengers in the fiscal year 2017-18.
As there has been a steady growth in the domestic passengers, the renovation work of Terminal-1, which earlier handled international operations, with most modern facilities, was starte

2018-12-05 23:40:03,582 INFO sqlalchemy.engine.base.Engine COMMIT
Delhi CM launches free pilgrimage scheme for senior citizens

SUMMARY
------------------------------
Delhi Chief Minister Arvind Kejriwal Wednesday launched the ‘Mukhyamantri Teerth Yatra Yojana’ under which 1,100 senior citizens from each of the 70 assembly constituencies here can undertake free pilgrimage.
Under the scheme, a city resident above 60 years will be eligible to take free pilgrimage along with his or her spouse.
“The scheme was launched with an aim that every elderly person’s desire to undertake a pilgrimage in his life gets fulfilled.
Employees of central, state and local government or autonomous bodies are not eligible for the scheme.
The Delhi government has also signed a Memorandum of Understanding with the railways to cater to the pilgrims availing the scheme.

2018-12-05 23:40:04,091 INFO sqlalchemy.engine.base.Engine INSERT INTO news ("paperName", title, summary) VALUES (?, ?, ?)
2018-12-05 23:40:04,

2018-12-05 23:40:06,458 INFO sqlalchemy.engine.base.Engine ('Hindu Business Line', 'J Jayalalithaa remembered on second death anniversary', 'While the respective factions led by Palaniswami and Panneerselvam later merged on August 21, a party general council in September last year sacked Sasikala as its chief.')
2018-12-05 23:40:06,461 INFO sqlalchemy.engine.base.Engine COMMIT
BJP will make Rajasthan prosperous, says Amit Shah

SUMMARY
------------------------------
The BJP transformed poll-bound Rajasthan from a ‘bimaru’ to a developed state and would now make it prosperous, party president Amit Shah said here on Wednesday, while predicting that his party would form a government with full majority.
“While the Congress promoted dynastic politics, making members of one family party presidents, the BJP gives importance to those who perform and work to push the country ahead,” he said.
“The BJP, in contrast, practised the politics of development and got a good response,” he said.
The Cong